In [1]:
import random
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from generator import Generator

In [3]:
def plot():
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plot()

def pre_process_isotherm(isotherm):
    isotherm -= min(isotherm)
    isotherm /= max(isotherm)
    return isotherm
    
def load_dataset(path, interp=False, gen_silica=None, gen_carbon=None):
    min_exp_pressure_i = 40
    max_exp_pressure_i = 458
    with open(path, 'rb') as f:
            dataset = np.load(f)
            isotherm_data = dataset["isotherm_data"]
            pore_distribution_data = dataset["pore_distribution_data"]
    x = np.empty((isotherm_data.shape[0], (-min_exp_pressure_i + max_exp_pressure_i)))
    y = np.empty(pore_distribution_data.shape)
    for i in range(len(isotherm_data)):
        if interp:
            interp_isotherm = np.interp(gen_silica.pressures_s, gen_carbon.pressures_s, isotherm_data[i])
        else:
            interp_isotherm = isotherm_data[i]
        isotherm = pre_process_isotherm(interp_isotherm[min_exp_pressure_i:max_exp_pressure_i])
        pore_distribution = pre_process_isotherm(pore_distribution_data[i])
        x[i] = isotherm
        y[i] = pore_distribution
    #x, y = shuffle(x, y)
    return x, y

def create_model(input_shape):
    ## CONV NET
    # model = Sequential()
    # model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(input_shape,1)))
    # model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    # model.add(layers.Dropout(0.5))
    # model.add(layers.MaxPooling1D(pool_size=2))
    # model.add(layers.Flatten())
    # model.add(layers.Dense(100, activation='relu'))
    # model.add(layers.Dense(2, activation='softmax'))
    ###
        ## DENSE NET
    model = Sequential(
        [
            Input(shape=len(x_train[0],)),
            layers.Dense(400, activation='relu'),
            layers.Dense(200, activation='relu'),
            layers.Dense(100, activation='relu'),
            layers.Dense(50, activation='relu'),
            layers.Dense(2, activation='softmax')
        ]
    )
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [4]:
gen_silica = Generator(path_s="data/kernel_generated2/Kernel_s_Silica-loc-isoth1.xlsx.npy",
                              path_d="data/kernel_generated2/Kernel_d_Silica-loc-isoth1.xlsx.npy",
                              path_p_d="data/kernel_generated2/Pressure_d_Silica-loc-isoth1.xlsx.npy",
                              path_p_s="data/kernel_generated2/Pressure_s_Silica-loc-isoth1.xlsx.npy",
                              path_a="data/kernel_generated2/Size_Silica-loc-isoth1.xlsx.npy"
                              )
gen_carbon = Generator(path_s="data/initial kernels/Kernel_Carbon_Adsorption.npy",
                              path_d="data/initial kernels/Kernel_Carbon_Desorption.npy",
                              path_p_d="data/initial kernels/Pressure_Carbon.npy",
                              path_p_s="data/initial kernels/Pressure_Carbon.npy",
                              path_a="data/initial kernels/Size_Kernel_Carbon_Adsorption.npy"
                              )
x1, y1 = load_dataset('data/datasets/Carbon_classification.npz', interp=True, gen_carbon=gen_carbon, gen_silica=gen_silica)
x2, y2 = load_dataset('data/datasets/Silica_classification.npz', interp=False)

In [7]:
i = random.randint(0, len(x1))
plt.plot(x1[i], marker=".", label="Carbon isotherm")
plt.plot(x2[i], marker=".", label="Silica isotherm")
plt.plot(gen_carbon.a_array, y1[i], marker=".", label="Carbon distribution")
plt.plot(gen_silica.a_array, y2[i], marker=".", label="Silica distribution")
plot()

In [18]:
y_carbon = np.empty(shape = (len(x1), 2))
y_silica = np.empty(shape = (len(x1), 2))
for i in range(len(x1)):
    y_carbon[i] = np.array([0, 1])
    y_silica[i] = np.array([1, 0])

In [19]:
x = np.concatenate((x1, x2), axis=0)
y = np.concatenate((y_carbon, y_silica), axis=0)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [21]:
model = create_model(input_shape=len(x[0]))

In [27]:
mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/classification.keras', save_best_only=True,
                                           monitor='accuracy', mode='max', verbose=1, save_weights_only=False,
                                           save_freq='epoch')

reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                   patience=100, verbose=1, mode='auto')
history = model.fit(np.array(x_train), np.array(y_train),
                    epochs=10, batch_size=5000, shuffle=True,
                    validation_data=(np.array(x_test), np.array(y_test)), callbacks=[mcp_save, reduce_lr_loss])
plot_loss(history)

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 0.2316 - accuracy: 0.9447
Epoch 1: accuracy improved from -inf to 0.94474, saving model to data/models\classification.keras
10/10 [==============================] - 0s 39ms/step - loss: 0.2316 - accuracy: 0.9447 - val_loss: 0.1820 - val_accuracy: 0.9705 - lr: 0.0010
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 0.2705 - accuracy: 0.8978
Epoch 2: accuracy did not improve from 0.94474
10/10 [==============================] - 0s 27ms/step - loss: 0.2705 - accuracy: 0.8978 - val_loss: 0.1561 - val_accuracy: 0.9677 - lr: 0.0010
Epoch 3/10
10/10 [==============================] - ETA: 0s - loss: 0.2008 - accuracy: 0.9305
Epoch 3: accuracy did not improve from 0.94474
10/10 [==============================] - 0s 26ms/step - loss: 0.2008 - accuracy: 0.9305 - val_loss: 0.1288 - val_accuracy: 0.9753 - lr: 0.0010
Epoch 4/10
10/10 [==============================] - ETA: 0s - loss: 0.1356 - accuracy: 0.9692
Epo

In [28]:
# Test on experimental data
exp_file = "MCM-41-adsorb"
exp_data = np.loadtxt(f"data/real/{exp_file}.txt")
p_exp = np.empty(len(exp_data))
n_exp = np.empty(len(exp_data))
for i in range(len(exp_data)):  # Загружаем адсорбцию и давления
    p_exp[i] = exp_data[i][0]
    n_exp[i] = exp_data[i][1]
j = np.argmax(n_exp) + 1  # Отделяем адсорбционную ветку
n_s_exp_raw = n_exp[:j]
p_exp_sorb = p_exp[:j]
plt.plot(p_exp_sorb, n_s_exp_raw, marker=".", label="MCM-41")
plot()

In [29]:
# интерполируем экспериментальную изотерму под давления кернала
n_s_exp_uncut = np.interp(gen_silica.pressures_s[40:], p_exp_sorb, n_s_exp_raw)
plt.plot(gen_silica.pressures_s[40:], n_s_exp_uncut)
n_s_exp = n_s_exp_uncut

In [30]:
n_s_exp_for_net = pre_process_isotherm(n_s_exp)
fit_exp = model.predict(np.array([n_s_exp_for_net])).T

1/1 [==============================] - 0s 23ms/step


In [31]:
fit_exp

array([[0.03094007],
       [0.9690599 ]], dtype=float32)